In [3]:
from konlpy.tag import Okt
import pandas as pd
import random

# Okt 형태소 분석기 초기화
okt = Okt()

def augment_korean_text(text, num_augments=3):
    # 형태소 분석
    morphs = okt.pos(text, norm=True, stem=True)
    augmented_texts = []
    
    for _ in range(num_augments):
        new_text = []
        for word, pos in morphs:
            # 형용사(Adjective)와 동사(Verb)에 대한 유의어 처리
            if pos in ['Adjective', 'Verb']:
                # 예시 유의어 사전 (실제 구현시 더 확장 필요)
                synonyms = {
                        # 맛 관련
                        '달달한': ['달콤한', '달달한', '달콤달콤한', '스위트한', '달큰한'],
                        '부드러운': ['순한', '부드러운', '순하디순한', '소프트한', '매끄러운', '부드럽디부드러운'],
                        '강한': ['독한', '강력한', '세', '독하디독한', '알콜도수가 높은', '도수가 강한'],
                        '상큼한': ['새콤한', '산뜻한', '신선한', '프레시한', '시원상큼한', '깔끔한'],
                        '쓴': ['씁쓸한', '쌉쌀한', '쓰디쓴', '씁슬한', '쌉싸름한', '쌉쓰름한'],
                        '신맛': ['새콤한 맛', '산미', '시큼한 맛', '새콤달콤한'],
                        
                        # 행동 관련
                        '마시': ['들이켜', '마셔', '음미하', '맛보', '즐기', '홀짝'],
                        '찾': ['구하', '원하', '찾아보', '알아보', '검색하'],
                        '추천': ['소개', '알려주', '추천', '제안', '안내', '설명'],
                        '싶': ['원하', '바라', '고프', '희망하', '기대하'],
                        
                        # 술 관련 표현
                        '도수': ['알콜 도수', '알코올 농도', '알코올 함량'],
                        '술': ['음료', '주류', '알코올', '칵테일'],
                        
                        # 정도 표현
                        '매우': ['무척', '아주', '너무', '엄청', '굉장히'],
                        '약간': ['조금', '살짝', '다소', '약간'],
                        '적당한': ['알맞은', '중간 정도의', '적절한', '딱 좋은'],
                        
                        # 선호도 표현
                        '좋아하': ['선호하', '즐기', '취향이'],
                        '필요하': ['요구되', '찾고 있', '구하고 싶'],
                        
                        # 기분/상황 관련
                        '기분 좋은': ['즐거운', '행복한', '기분전환', '기분전환되는'],
                        '분위기 있는': ['로맨틱한', '특별한', '고급스러운']
                    }
                
                if word in synonyms:
                    new_text.append(random.choice(synonyms[word]))
                else:
                    new_text.append(word)
            else:
                new_text.append(word)
        
        augmented_texts.append(' '.join(new_text))
    
    return augmented_texts

# 데이터프레임 생성
data = {
    "입력 문장": [
        "나는 달달한 칵테일을 마시고 싶어.",
        "도수가 낮은 상큼한 칵테일을 추천해줘.",
        "강한 도수의 쓴맛 나는 보드카 추천해줘.",
        "부드럽고 달달한 위스키를 마셔보고 싶어.",
        "상큼하고 도수가 낮은 럼주를 소개해줘.",
        "도수가 중간이고 달달한 보드카가 있나요?",
        "신맛이 나고 도수가 강한 술을 찾고 있어요.",
        "도수가 약하고 부드러운 칵테일 추천 부탁해요.",
        "쓴맛 나는 위스키를 마셔보고 싶어.",
        "달달하고 상큼한 럼을 마시고 싶어요.",
        "가볍고 부드러운 칵테일이 필요해요.",
        "진하고 쓴 럼을 추천해줄래?",
        "부드럽고 달달한 술이 있었으면 좋겠어요.",
        "상큼한 맛이 나는 도수가 높은 보드카를 마셔보고 싶어요.",
        "쓴맛이 강한 위스키를 소개해주세요.",
        "도수가 낮고 달달한 술을 마시고 싶어요.",
        "도수가 높은 럼 중에서 신맛 나는 술을 찾고 있어요.",
        "가볍고 상큼한 칵테일 추천 부탁드립니다.",
        "쓴맛이 적고 부드러운 보드카를 원해요.",
        "달달한 맛이 나는 위스키를 찾고 있어요.",
        "신맛이 강하고 도수가 낮은 술 추천 부탁드려요.",
        "강한 도수의 부드러운 위스키를 추천해줘.",
        "쓴맛과 상큼함이 공존하는 칵테일이 있나요?",
        "달달하면서 도수가 높은 보드카를 찾고 있어요.",
        "가볍고 신맛 나는 럼을 추천해주세요."
    ],
    "도수": [
        "중간", "낮은", "높은", "높은", "낮은",
        "중간", "높은", "낮은", "높은", "중간",
        "낮은", "높은", "중간", "높은", "높은",
        "낮은", "높은", "낮은", "중간", "높은",
        "낮은", "높은", "중간", "높은", "낮은"
    ],
    "술 종류": [
        "칵테일", "칵테일", "보드카", "위스키", "럼",
        "보드카", "칵테일", "칵테일", "위스키", "럼",
        "칵테일", "럼", "칵테일", "보드카", "위스키",
        "칵테일", "럼", "칵테일", "보드카", "위스키",
        "칵테일", "위스키", "칵테일", "보드카", "럼"
    ],
    "맛": [
        "달달한", "상큼한", "쓴맛", "부드러운", "상큼한",
        "달달한", "신맛", "부드러운", "쓴맛", "달달한",
        "부드러운", "쓴맛", "달달한", "상큼한", "쓴맛",
        "달달한", "신맛", "상큼한", "부드러운", "달달한",
        "신맛", "부드러운", "쓴맛", "달달한", "신맛"
    ]
}
df = pd.DataFrame(data)

# 데이터 증강
augmented_data = {
    "입력 문장": [],
    "도수": [],
    "술 종류": [],
    "맛": []
}

# 각 문장에 대해 증강 수행
for idx, row in df.iterrows():
    # 원본 데이터 추가
    augmented_data["입력 문장"].append(row["입력 문장"])
    augmented_data["도수"].append(row["도수"])
    augmented_data["술 종류"].append(row["술 종류"])
    augmented_data["맛"].append(row["맛"])
    
    # 증강 데이터 생성 및 추가
    augmented_texts = augment_korean_text(row["입력 문장"])
    for aug_text in augmented_texts:
        augmented_data["입력 문장"].append(aug_text)
        augmented_data["도수"].append(row["도수"])
        augmented_data["술 종류"].append(row["술 종류"])
        augmented_data["맛"].append(row["맛"])

# 결과를 데이터프레임으로 변환
augmented_df = pd.DataFrame(augmented_data)
augmented_df.to_csv("data/augmented_data.csv", index=False)

ValueError: jpype jar must be ascii to add to the system class path

: 